# 1

In [38]:
#gym
import gymnasium as gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers
import numpy as np
import random
import os

#stableblines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2

In [16]:
Discrete(3).sample()

2

In [21]:
Box(0,1, shape=(3,3)).sample()

array([[0.263976  , 0.19165584, 0.55274254],
       [0.95086247, 0.01216521, 0.13791534],
       [0.36554003, 0.964149  , 0.18097499]], dtype=float32)

In [25]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(0, array([0.22451706, 0.92080486, 0.75349087], dtype=float32))

In [26]:
Dict({'height':Discrete(6), 'speed':Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([71.87945], dtype=float32))])

In [29]:
MultiBinary(4).sample()

array([0, 0, 0, 1], dtype=int8)

In [35]:
MultiDiscrete([5,2,2]).sample()

array([2, 1, 0], dtype=int64)

# 3

In [39]:
Box(low=np.array([0]), high=np.array([100]))

C:\Users\nurma\TestEnvOne\EnvOne\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Box(0.0, 100.0, (1,), float32)

In [40]:
Box(0,100, shape=(1,))

Box(0.0, 100.0, (1,), float32)

In [43]:
np.array([38+random.randint(-3,3)]).astype(float)

array([37.])

In [60]:
class ShowerEnv(Env):
    def __init__(self): #sets up the env when first run?
        self.action_space = Discrete(3) # tap up, down, or no action
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action): #
        #apply temp
        self.state += action-1

        #decreate time
        self.shower_length -= 1

        #calc reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <=0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render(self): #renders env
        #not going to implement this
        pass
    def reset(self): #resets temp and shower time
        self.state = 38 + random.randint(-3,3)#idk why tf but u can do this too??? np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [61]:
env = ShowerEnv()

In [62]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [63]:
env.action_space

Discrete(3)

In [75]:
env.reset()

39

In [76]:
env.step(2)

(40, -1, False, {})

# 4

In [80]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-22
Episode:2 Score:-40
Episode:3 Score:-24
Episode:4 Score:-60
Episode:5 Score:-42


# 5

In [81]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\nurma\TestEnvOne\EnvOne\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [84]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_15
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 19       |
| time/              |          |
|    fps             | 396      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 13.3        |
| time/                   |             |
|    fps                  | 319         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007683823 |
|    clip_fraction        | 0.127       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.791      |
|    explained_variance   | 0.0329      

# 6

In [87]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [88]:
model.save(shower_path)

In [89]:
del model

In [91]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\nurma\TestEnvOne\EnvOne\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [92]:
evaluate_policy(model, env, n_eval_episodes=200)

C:\Users\nurma\TestEnvOne\EnvOne\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(23.68, 54.78592519981752)